# LangChain Expression Language (LCEL)

This notebook demonstrates the **LangChain Expression Language (LCEL)**,
which enables composing prompt templates, models, and output parsers
into a single executable pipeline.

The focus is on:
- Building prompt templates with structured instructions
- Parsing model output into deterministic formats
- Composing components using the pipe (`|`) operator
- Executing end-to-end chains in a single expression

LCEL provides a concise and readable way to define LLM workflows.


In [ ]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Output Format Instructions

Before building the prompt, we define formatting instructions
using an output parser.

These instructions are injected into the prompt to ensure
the model returns a predictable, structured response.


In [ ]:
list_intructions =  CommaSeparatedListOutputParser().get_format_instructions()
list_intructions

## Model Initialization

A deterministic chat model is initialized to ensure
consistent and parseable outputs when chaining components.


In [ ]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'},"reasoning":{"effort": "medium"}},
    
    ) 

## Prompt Template Construction

A `ChatPromptTemplate` is created with a dynamic placeholder
and embedded formatting instructions.

This template defines the structure of the user request
without executing the model.


In [ ]:
chat_template = ChatPromptTemplate.from_messages([('human', "I've recently adopted a {pet}, Can you suggest three {pet} names? \n" + list_intructions)])
chat_template

## Manual Invocation and Parsing

Before using LCEL, the prompt, model, and output parser
can be invoked step by step to observe intermediate results.


In [ ]:
list_output_parser =  CommaSeparatedListOutputParser()

In [ ]:
chat_template_result = chat_template.invoke({"pet": "dog"})

In [ ]:
chat_result = chat.invoke(chat_template_result)

In [ ]:
list_output_parser.invoke(chat_result)

## Composing Chains with LCEL

Using LCEL, individual components can be composed into a single
executable chain using the pipe (`|`) operator.

This allows the entire workflow to be expressed concisely
in one readable line.


In [ ]:
chain = chat_template | chat | list_output_parser   # Create a chain by piping components together using expression language, all of the above process in one line
chain.invoke({"pet": "cat"})

## Types of LangChain Objects That Can Be Integrated into a Chain

- **Runnable** is the core abstraction in LangChain.  
  A Runnable represents a unit of work that can be invoked, batched,
  streamed, transformed, and composed with other Runnables.

- **ChatPromptTemplate**, **Models**, and **Output Parsers**
  are all implemented as Runnables.

- **RunnableSequence** represents an ordered composition of Runnables.
  Chains created with LCEL are RunnableSequences.

- Since a RunnableSequence is itself a Runnable, chains can be composed
  together to form longer and more complex pipelines.


In [ ]:

type(chain)
type(chat_template)

## RunnablePassthrough

`RunnablePassthrough` acts as an identity Runnable.
It forwards inputs without modification and is useful
for wiring values between different parts of a chain.


In [ ]:
RunnablePassthrough().invoke("This is a test string.")

In [ ]:
RunnablePassthrough().invoke([1, 2, 3, 4, 5])

## Chaining Chains Together

Chains are themselves Runnables and can be composed
to form longer pipelines.

This enables multi-step reasoning workflows.


In [ ]:
chat_template_tools = ChatPromptTemplate.from_template('''
    What are the five most import tools a {job title} needs?
    Answer only by listing the tools as a numbeered list
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
    Condisering the tools provided, develop a strategy for effectively learning and mastering them:
    {tools}
''')

In [ ]:
chat_template_tools

In [ ]:
string_parser = StrOutputParser()

In [ ]:

chain_strategy = chat_template_strategy | chat | string_parser
chain_tools =  chat_template_tools | chat | string_parser | {'tools': RunnablePassthrough()}

In [ ]:
print(chain_tools.invoke({"job title": "data scientist"}))

In [ ]:
print(chain_strategy.invoke({'tools': '''
    1. Python
    2. SQL
    3. R
    4. Jupyter Notebook
    5. Git
'''}))

## Combining Chains into a Single Pipeline

Previously defined chains can be composed together
to form a complete end-to-end workflow.


In [ ]:
chain_combined = chain_tools | chain_strategy
print(chain_combined.invoke({"job title": "data scientist"}))

## Long-Form LCEL Expression

The same workflow can be written as a single LCEL expression.


In [ ]:
chain_long = (chat_template_tools | chat | string_parser | {'tools': RunnablePassthrough()} | 
chat_template_strategy | chat | string_parser)

In [ ]:
print(chain_long.invoke({"job title": "data scientist"}))

## Visualizing Chain Execution Graphs

LangChain provides a built-in way to visualize the execution graph
of a chain. This visualization is powered internally by the
**`grandalf`** graph layout library.

The graph representation helps:
- Understand the execution order of Runnables
- Inspect how data flows between components
- Debug complex LCEL pipelines
- Reason about parallel and sequential execution

Each node in the graph represents a Runnable, and edges represent
data flow between them.


In [ ]:
chain_long.get_graph().print_ascii()

## RunnableParallel

`RunnableParallel` allows multiple Runnables to execute
concurrently using the same input values.


In [ ]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [ ]:
chain_books = chat_template_books | chat | string_parser
chain_projects = chat_template_projects | chat | string_parser

In [ ]:
chain_parallel = RunnableParallel({'books': chain_books, 'projects': chain_projects})

In [ ]:
chain_parallel.invoke({'programming language': 'Python'})

## RunnableParallel Graph


In [ ]:
chain_parallel.get_graph().print_ascii()

## Batch vs RunnableParallel

- **Batch**: Same Runnable, multiple different inputs  
- **RunnableParallel**: Different Runnables, same input  

RunnableParallel is ideal when multiple tasks depend
on the same input context.


### Explicit vs Implicit `RunnableParallel` Syntax in LCEL

LangChain Expression Language (LCEL) supports **two equivalent syntaxes**
for parallel execution:

1. **Explicit syntax** using `RunnableParallel`
2. **Implicit syntax** using a dictionary of Runnables

#### Explicit syntax
```python
RunnableParallel({'books': chain_books, 'projects': chain_projects})


In [ ]:
chain_time1 = (RunnableParallel({'books': chain_books, 'projects': chain_projects})
                                | chat_template_time 
                                | chat
                                | string_parser
                                )

In [ ]:
chain_time2 = ({'books': chain_books, 'projects': chain_projects}
                                | chat_template_time 
                                | chat
                                | string_parser
                                )     #this 2nd version without RunnableParallel but automatically parallelizes

In [ ]:
print(chain_time1.invoke({'programming language': 'Python'}))

In [ ]:
print(chain_time2.invoke({'programming language': 'Python'}))

## Final Chain Graphs

In [ ]:
chain_time1.get_graph().print_ascii()

In [ ]:
chain_time2.get_graph().print_ascii()

## Summary

This notebook demonstrated:

- Defining structured output instructions  
- Creating reusable chat prompt templates  
- Parsing model output into deterministic formats  
- Composing prompt, model, and parser using LCEL  
- Chaining chains together into longer pipelines  
- Using RunnablePassthrough and RunnableParallel  
- Visualizing execution graphs  

LCEL enables expressive, composable, and scalable
LLM application design.
